In [1]:
import os
import cv2 as cv
import numpy as np
import shutil
from shutil import copy

In [2]:
os.makedirs('yolov5-master/test')
os.makedirs('yolov5-master/test/images')
os.makedirs('yolov5-master/test/labels')

In [3]:
base_dir = r'Training_Dataset'
images_dir = r'test/images'
labels_dir = r'test/labels'

In [4]:
annotation = os.path.join(base_dir,'annotation.txt')

In [5]:
ImageName = []
Label = []

with open(annotation,'r') as f:
    data = f.readlines()
    for line in data:
        sp = line.split(',',1)
        ImageName.append(sp[0])
        Label.append(sp[1])

In [6]:
del ImageName[0]
del Label[0]

# Image

In [7]:
a = os.listdir(base_dir)
b ={}
for i in range(6):
    b[i] = os.listdir(base_dir+'/'+a[i])

In [8]:
for j in range(len(a)-1):
    for k in range(len(b[j])):
        orgin = base_dir+'/'+a[j]+'/'+b[j][k]
        copy = images_dir+'/'+a[j]+'-'+b[j][k]
        shutil.copy(orgin, copy)

In [9]:
imagefile = {}
H = {}
W = {}

for i in range(len(ImageName)):
    imagefile[i] = cv.imread(ImageName[i])
    (H[i],W[i]) = imagefile[i].shape[:2]

# Label

In [10]:
Label[0],type(Label[0])

('0,2459,1274,2530,1329,0,1613,334,1679,396,0,1726,794,1797,854\n', str)

In [11]:
def labelnormalize(label,num):
    label = label.rstrip()
    label = label.split(',')
    
    for i in range(int(len(label)/5)):
        i=i*5
        Xmin = int(label[i+1])
        Ymin = int(label[i+2])
        Xmax = int(label[i+3])
        Ymax = int(label[i+4])
        
        label[i+1]=(Xmin+Xmax)/(2*W[num])
        label[i+2]=(Ymin+Ymax)/(2*H[num])
        label[i+3]=(Xmax-Xmin)/W[num]
        label[i+4]=(Ymax-Ymin)/H[num]
        
    return label

In [12]:
for i in range(len(Label)):
    Label[i] = labelnormalize(Label[i],i)

In [13]:
def lablewrite(ac):
    a={}
    b=1
    
    ac = ' '.join([str(item) for item in ac])
    
    for i in range(len(ac)):
        if ac[i] == str(' '):
            a[b]=i
            b=b+1

    ac = list(ac)

    for i in range(1,b):
        if (i%5)==0:
            q=a[i]
            ac[q] = ('\n')

    ac = ''.join([str(item) for item in ac])
    
    return ac

In [14]:
l = 0

for j in range(len(a)-1):
    for k in range(len(b[j])):
        filename = labels_dir+'/'+str(j)+'-'+str(k)
        aa = lablewrite(Label[l])
        f1 = open(filename+'.txt','w')
        f1.write(aa)
        l=l+1
f1.close()